In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings 
import re 
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"C:\Users\takue\Documents\Data Science\Super Store\Sample Superstore - Orders.csv")
df.head()

,Order Priority,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,Product Sub-Category,Product Container,Product Name,Region,State or Province,City,Order Date,Ship Date,Quantity,Order ID
0,Not Specified,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,Labels,Small Box,Avery 49,Central,Illinois,Addison,28/05/2012,30/05/2012,2,88525
1,High,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Pens & Art Supplies,Wrap Bag,SANFORD Liquid Accent™ Tank-Style Highlighters,West,Washington,Anacortes,07/07/2010,08/07/2010,4,88522
2,Not Specified,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Paper,Small Box,Xerox 1968,West,Washington,Anacortes,27/07/2011,28/07/2011,7,88523
3,Not Specified,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,"Scissors, Rulers and Trimmers",Small Pack,Acme® Preferred Stainless Steel Scissors,West,Washington,Anacortes,27/07/2011,28/07/2011,7,88523
4,Not Specified,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,Telephones and Communication,Small Box,V70,West,Washington,Anacortes,27/07/2011,27/07/2011,8,88523


In [3]:
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce', yearfirst=True)
df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors='coerce', yearfirst=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9426 entries, 0 to 9425
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Order Priority        9426 non-null   object        
 1   Unit Price            9426 non-null   float64       
 2   Shipping Cost         9426 non-null   float64       
 3   Customer ID           9426 non-null   int64         
 4   Customer Name         9426 non-null   object        
 5   Ship Mode             9426 non-null   object        
 6   Customer Segment      9426 non-null   object        
 7   Product Category      9426 non-null   object        
 8   Product Sub-Category  9426 non-null   object        
 9   Product Container     9426 non-null   object        
 10  Product Name          9426 non-null   object        
 11  Region                9426 non-null   object        
 12  State or Province     9426 non-null   object        
 13  City              

In [4]:
df['SalesAmountWithShipping'] = df['Unit Price'] * df['Quantity '] + df['Shipping Cost']

In [5]:
cust_min = df.groupby('Customer ID')['Order Date'].min().reset_index()
cust_min.rename({'Order Date' : 'First Date'}, axis=1, inplace=True)
cust_min.head()

,Customer ID,First Date
0,2,2012-05-28
1,3,2010-07-07
2,5,2010-12-13
3,6,2011-05-26
4,7,2012-10-30


In [6]:
cust_max = df.groupby('Customer ID')['Order Date'].max().reset_index()
cust_max.rename({'Order Date' : 'Last Date'}, axis=1, inplace=True)
cust_max.head()

,Customer ID,Last Date
0,2,2012-05-28
1,3,2013-07-01
2,5,2012-05-12
3,6,2012-12-29
4,7,2012-10-30


In [7]:
cust = cust_min.merge(cust_max, how='inner', on='Customer ID')
cust.head()

,Customer ID,First Date,Last Date
0,2,2012-05-28,2012-05-28
1,3,2010-07-07,2013-07-01
2,5,2010-12-13,2012-05-12
3,6,2011-05-26,2012-12-29
4,7,2012-10-30,2012-10-30


In [8]:
# Calculate Customer Tenure (Number of Year one has been our customer)
cust['Customer Tenure'] = ((cust['Last Date'].max() - cust['First Date']).dt.days / 365).astype(int)
cust.head()

,Customer ID,First Date,Last Date,Customer Tenure
0,2,2012-05-28,2012-05-28,1
1,3,2010-07-07,2013-07-01,3
2,5,2010-12-13,2012-05-12,3
3,6,2011-05-26,2012-12-29,2
4,7,2012-10-30,2012-10-30,1


In [9]:
# Calculate Recency (days since last purchase)
cust['Recency'] = (cust['Last Date'].max() - cust['Last Date']).dt.days
cust.head()

,Customer ID,First Date,Last Date,Customer Tenure,Recency
0,2,2012-05-28,2012-05-28,1,582
1,3,2010-07-07,2013-07-01,3,183
2,5,2010-12-13,2012-05-12,3,598
3,6,2011-05-26,2012-12-29,2,367
4,7,2012-10-30,2012-10-30,1,427


In [10]:
# Calculate the Frequency of each customer 
cust_freq = df.groupby('Customer ID')['Order ID'].size().reset_index()
cust_freq.rename({'Order ID' : 'Frequency'}, axis=1, inplace=True)
cust_freq.head()

,Customer ID,Frequency
0,2,1
1,3,6
2,5,2
3,6,4
4,7,1


In [11]:
# Calculate the Total Revenue  for each customer 
cust_revenue = df.groupby('Customer ID')['SalesAmountWithShipping'].sum().reset_index()
cust_revenue.rename({'SalesAmountWithShipping' : 'Monetary'}, axis=1, inplace=True)
cust_revenue.head()

,Customer ID,Monetary
0,2,6.26
1,3,5303.98
2,5,6161.21
3,6,3625.09
4,7,238.18


In [12]:
cust = cust.merge(cust_freq, how='inner', on='Customer ID').merge(cust_revenue, how='inner', on='Customer ID')
cust.head()

,Customer ID,First Date,Last Date,Customer Tenure,Recency,Frequency,Monetary
0,2,2012-05-28,2012-05-28,1,582,1,6.26
1,3,2010-07-07,2013-07-01,3,183,6,5303.98
2,5,2010-12-13,2012-05-12,3,598,2,6161.21
3,6,2011-05-26,2012-12-29,2,367,4,3625.09
4,7,2012-10-30,2012-10-30,1,427,1,238.18


In [13]:
# Calculate the Average Purchase value of each customer
cust['Average Purchase Value'] = cust['Monetary'] / cust['Frequency']
cust.head()

,Customer ID,First Date,Last Date,Customer Tenure,Recency,Frequency,Monetary,Average Purchase Value
0,2,2012-05-28,2012-05-28,1,582,1,6.26,6.260000
1,3,2010-07-07,2013-07-01,3,183,6,5303.98,883.996667
2,5,2010-12-13,2012-05-12,3,598,2,6161.21,3080.605000
3,6,2011-05-26,2012-12-29,2,367,4,3625.09,906.272500
4,7,2012-10-30,2012-10-30,1,427,1,238.18,238.180000


In [14]:
# Calculate the customer lifetime value of each customer 
cust['Customer Lifetime Value'] = cust['Monetary'] * cust['Customer Tenure']
cust.head()

,Customer ID,First Date,Last Date,Customer Tenure,Recency,Frequency,Monetary,Average Purchase Value,Customer Lifetime Value
0,2,2012-05-28,2012-05-28,1,582,1,6.26,6.260000,6.26
1,3,2010-07-07,2013-07-01,3,183,6,5303.98,883.996667,15911.94
2,5,2010-12-13,2012-05-12,3,598,2,6161.21,3080.605000,18483.63
3,6,2011-05-26,2012-12-29,2,367,4,3625.09,906.272500,7250.18
4,7,2012-10-30,2012-10-30,1,427,1,238.18,238.180000,238.18


In [15]:
# Define thresholds for customer segments based on CLV
rfm = cust.copy()
high_value_threshold = rfm['Customer Lifetime Value'].quantile(0.75)  # Top 25% CLV
low_value_threshold = rfm['Customer Lifetime Value'].quantile(0.25)   # Bottom 25% CLV

# Assign customer segments
rfm['Customer Segment'] = pd.cut(rfm['Customer Lifetime Value'], 
                                  bins=[-float('inf'), low_value_threshold, high_value_threshold, float('inf')],
                                  labels=['Low Value', 'Medium Value', 'High Value'])

# Display the customer segmentation
rfm.head()


,Customer ID,First Date,Last Date,Customer Tenure,Recency,Frequency,Monetary,Average Purchase Value,Customer Lifetime Value,Customer Segment
0,2,2012-05-28,2012-05-28,1,582,1,6.26,6.260000,6.26,Low Value
1,3,2010-07-07,2013-07-01,3,183,6,5303.98,883.996667,15911.94,High Value
2,5,2010-12-13,2012-05-12,3,598,2,6161.21,3080.605000,18483.63,High Value
3,6,2011-05-26,2012-12-29,2,367,4,3625.09,906.272500,7250.18,Medium Value
4,7,2012-10-30,2012-10-30,1,427,1,238.18,238.180000,238.18,Medium Value


### Differentiate Spending Based on Customer Value

Losing a high-value customer is much more costly than losing a low value customer. Therefore, when allocating your retention budget think about different value-based segments and how likely they are to buy from you again. For this you will need to be able to calculate the likelihood to buy for each segment. The customers with a lower likelihood to buy are at greater risk of leaving you—and never make another purchase again.

In [16]:
def risk_allocation(latest_year):
    risk_data = cust.copy()

    # Ensure 'First Date' and 'Last Date' are datetime
    risk_data['First Date'] = pd.to_datetime(risk_data['First Date'])
    risk_data['Last Date'] = pd.to_datetime(risk_data['Last Date'])

    # Identify all past customers (who made their first purchase before the latest year)
    past_customers = risk_data[risk_data['First Date'].dt.year < latest_year].copy()

    # New customers (first purchase was in the latest year)
    new_customers = risk_data[risk_data['First Date'].dt.year == latest_year].copy()
    new_customers['Risk Level'] = 'New'

    # Retained customers (first purchase was before latest year & purchased in latest year)
    retained_customers = risk_data[
        (risk_data['First Date'].dt.year < latest_year) & 
        (risk_data['Last Date'].dt.year == latest_year)
    ].copy()

    # Apply Risk Level categorization for retained and past customers
    for df in [retained_customers, past_customers]:
        df['Risk Level'] = pd.cut(
            df['Average Purchase Value'], 
            bins=[-1, 110, 900, float('inf')], 
            labels=['Low', 'Med', 'High']
        )

    # Combine retained and new customers
    final_data = pd.concat([new_customers, retained_customers])

    # Aggregate customer counts per risk level
    customer_count = final_data.groupby('Risk Level')['Customer ID'].nunique()

    # Retention = (Retained customers in latest year) / (Total past customers in the same risk level)
    retention = (
        retained_customers.groupby('Risk Level')['Customer ID'].nunique() /
        past_customers.groupby('Risk Level')['Customer ID'].nunique()
    ).reindex(customer_count.index).fillna(0) * 100  # Handle missing risk levels

    # Compute $ at Risk = (Number of Customers * Retention * AOV)
    aov = final_data.groupby('Risk Level')['Average Purchase Value'].mean()
    at_risk_value = (customer_count * (retention / 100) * aov).fillna(0)

    # Compute Risk Allocation (% of total risk)
    risk_allocation_perc = (at_risk_value / at_risk_value.sum()).fillna(0) * 100

    # Create final summary DataFrame
    summary_df = pd.DataFrame({
        'Number of Customers': customer_count,
        f'{latest_year} AOV': aov,
        'Retention (%)': retention,
        '$ at Risk': at_risk_value,
        'Risk Allocation (%)': risk_allocation_perc
    }).fillna(0)  

    # Format the DataFrame
    summary_df[f'{latest_year} AOV'] = summary_df[f'{latest_year} AOV'].apply(lambda x: f"${x:,.2f}")
    summary_df['Retention (%)'] = summary_df['Retention (%)'].apply(lambda x: f"{x:.2f}%")
    summary_df['$ at Risk'] = summary_df['$ at Risk'].apply(lambda x: f"${x:,.2f}")
    summary_df['Risk Allocation (%)'] = summary_df['Risk Allocation (%)'].apply(lambda x: f"{x:.2f}%")

    return summary_df

In [17]:
risk_allocation(2013)

,Number of Customers,2013 AOV,Retention (%),$ at Risk,Risk Allocation (%)
Risk Level,,,,,
High,393,"$2,053.63",61.70%,"$497,929.13",73.74%
Low,92,$73.01,22.83%,"$1,533.39",0.23%
Med,707,$445.11,55.85%,"$175,742.42",26.03%
New,397,$833.35,0.00%,$0.00,0.00%


### Find Products That Bring High-Value Customers